# Capstone Project - The Battle of the Neighborhoods THE BRAZIL VERSION
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project, we will try to find an ideal place for a restaurant. Specifically, this report will be directed to interested parties interested in opening a ** izzeria** in **Madureira**, Rio de Janeiro, Brazil.

As there are many restaurants in Rio, we will try to detect **places that are not yet full of restaurants**. We are also particularly interested in **reas without pizzerias nearby**. We would also prefer locations **as close as possible to the chosen center**, assuming the first two conditions are met.

We will use our data science powers to generate some more promising neighborhoods based on these criteria. The advantages of each area will then be clearly expressed so that the best possible final location can be chosen by the interested parties.

## Data <a name="data"></a>

Based on the definition of our problem, the factors that will influence our decision are:
* number of restaurants in the neighborhood (any type of restaurant)
* number and distance of pizzerias in the neighborhood, if any
* distance from the neighborhood to the center

We decided to use a grid of places with regular spacing, centered in the center of the city, to define our neighborhoods.

The following data sources will be needed to extract / generate the necessary information:
* candidate area centers will be generated algorithmically and approximate addresses of centers in those areas will be obtained using **reverse geocoding from the Google Maps API**
* number of restaurants and their type and location in each neighborhood will be obtained using the **Foursquare API**
* the coordinate of the center of Madureira will be obtained using **geocoding from the Google Maps API**

### Neighborhood Candidates

Let's create latitude and longitude coordinates for the centroid of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is approx. 12x12 killometers centered in the center of Madureira.

Let's first find the latitude and longitude of the center, using a specific and known address and the Google Maps geocoding API.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import requests
from geopy.geocoders import Nominatim

google_api_key = ''

In [2]:
address = 'Madureira, Rio de Janeiro, Brasil'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
rio_center = [lat,lon]
    
print('Coordinate of {}: {}'.format(address, rio_center))

Coordinate of Madureira, Rio de Janeiro, Brasil: [-22.871658, -43.3371976]


Now we are going to create a grid of candidate areas, equally spaced, centered in the center of the city and within approximately 6 km. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be separated by 600 meters.

To calculate distances accurately, we need to create our grid of locations in the 2D Cartesian coordinate system, which allows us to calculate distances in meters (not in degrees of latitude / longitude). Next, we will project these coordinates in degrees of latitude / longitude to be shown on the Folium map. So, let's create functions to convert between the WGS84 spherical coordinate system (degrees of latitude / longitude) and the UTM Cartesian coordinate system (X / Y coordinates in meters).

In [3]:
#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Rio center longitude={}, latitude={}'.format(rio_center[1], rio_center[0]))
x, y = lonlat_to_xy(rio_center[1], rio_center[0])
print('Rio center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Rio center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Rio center longitude=-43.3371976, latitude=-22.871658
Rio center UTM X=-6240853.623284783, Y=-4309784.974270792
Rio center longitude=-43.33719760000196, latitude=-22.871657999999623


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
rio_center_x, rio_center_y = lonlat_to_xy(rio_center[1], rio_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = rio_center_x - 6000
x_step = 600
y_min = rio_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(rio_center_x, rio_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [5]:
#!pip install folium
import folium

In [6]:
map_rio = folium.Map(location=rio_center, zoom_start=13)
folium.Marker(rio_center, popup='Alexanderplatz').add_to(map_rio)
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rio) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_rio)
map_rio

Now we have the coordinates of the centers of the neighborhoods / areas to be evaluated, equally spaced (the distance from each point to its neighbors is exactly the same) and within approximately 6km from Madureira

Now, let's use the Google Maps API to get approximate addresses for these locations.

In [7]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, rio_center[0], rio_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(rio_center[0], rio_center[1], addr))

Reverse geocoding check
-----------------------
Address of [-22.871658, -43.3371976] is: None


In [8]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Brasil', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [9]:
addresses[150:170]

['NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS']

Api presented a problem and we were unable to obtain the address data, even so, we can continue with our code and later use the geocoder to obtain them.

In [10]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,NO ADDRESS,-22.893220,-43.364895,-6.242654e+06,-4.315501e+06,5992.495307
1,NO ADDRESS,-22.895024,-43.361841,-6.242054e+06,-4.315501e+06,5840.376700
2,NO ADDRESS,-22.896828,-43.358786,-6.241454e+06,-4.315501e+06,5747.173218
3,NO ADDRESS,-22.898631,-43.355730,-6.240854e+06,-4.315501e+06,5715.767665
4,NO ADDRESS,-22.900435,-43.352675,-6.240254e+06,-4.315501e+06,5747.173218
5,NO ADDRESS,-22.902239,-43.349619,-6.239654e+06,-4.315501e+06,5840.376700
6,NO ADDRESS,-22.904043,-43.346563,-6.239054e+06,-4.315501e+06,5992.495307
7,NO ADDRESS,-22.888065,-43.367791,-6.243554e+06,-4.314981e+06,5855.766389
8,NO ADDRESS,-22.889868,-43.364737,-6.242954e+06,-4.314981e+06,5604.462508
9,NO ADDRESS,-22.891671,-43.361682,-6.242354e+06,-4.314981e+06,5408.326913


...and let's now save/persist this data into local file.

In [11]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use the Foursquare API to get information about restaurants in each neighborhood.

We are interested in places in the 'food' category, but only those that are suitable restaurants - coffee shops, pizzerias, bakeries etc. are not direct competitors, so we don't care about them. Therefore, we will include in our list only those places that have 'restaurant' in the category name and will make sure to detect and include all subcategories of the specific category of 'pizzeria', as we need information about pizzerias in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.

In [12]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

pizza_categories = '4bf58dd8d48988d1ca941735'

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Rio de Janeiro', '')
    address = address.replace(', Brasil', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [13]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found pizza restaurants
import pickle

foursquare_client_id = 'TVNRIUWD32PUD3NFNIJBYBLFHDO0KYAD1TTKDVDIXZN4ILAA' # your Foursquare ID
foursquare_client_secret = '5RACCOV3NE3BMRUNNLC3HGYS5MWZGOPTO53BRHRYMU3EZOMC' # your Foursquare Secret
VERSION = '20180605'

def get_restaurants(lats, lons):
    restaurants = {}
    pizza_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_pizza = is_restaurant(venue_categories, specific_filter=pizza_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_pizza, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_pizza:
                    pizza_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, pizza_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
pizza_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('pizza_restaurants_350.pkl', 'rb') as f:
        pizza_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, pizza_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('pizza_restaurants_350.pkl', 'wb') as f:
        pickle.dump(pizza_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [14]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Pizza restaurants:', len(pizza_restaurants))
print('Percentage of Pizza restaurants: {:.2f}%'.format(len(pizza_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 247
Total number of Pizza restaurants: 62
Percentage of Pizza restaurants: 25.10%
Average number of restaurants in neighborhood: 1.2417582417582418


In [15]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4de985a51fc7d5135ca89670', 'Sorvetão Restaurante e Pizzaria', -22.89832460311299, -43.35293369749919, 'R. Candido Benicio, 1970, RJ, 21321-801', 144, False, -6240492.486822234, -4315205.631005052)
('51e1fe8a498ee22b4c888b26', 'Bosquinho da Praça', -22.90060823858688, -43.35401016800587, 'Praça Seca, RJ', 138, False, -6240423.482696499, -4315645.672716209)
('4fc638017b0c7e019f026728', "Pizoni's Restaurante e Pizzaria", -22.901826038697497, -43.35159325355559, 'R. Florianópolis, 1481, RJ, 21321-050', 207, True, -6239968.985263123, -4315614.173286992)
('4d989f49b188721e56f72b37', "Pizoni's", -22.901800030699295, -43.35159634603115, 'R. Florianópolis, 1481, 21321-050', 208, True, -6239971.9150914885, -4315610.528102775)
('53aeb1b0498e5522d44a0226', 'Açougue Barão', -22.89767354607587, -43.35128598726851, 'Brasil', 313, False, -6240324.887366496, -4314960.870628703)
('4f47d9d8e4b024e50b761712', 'Pizzonis', -22.90620630501871, -43.34803845646

In [16]:
print('List of Pizza restaurants')
print('---------------------------')
for r in list(pizza_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(pizza_restaurants))

List of Pizza restaurants
---------------------------
('4fc638017b0c7e019f026728', "Pizoni's Restaurante e Pizzaria", -22.901826038697497, -43.35159325355559, 'R. Florianópolis, 1481, RJ, 21321-050', 207, True, -6239968.985263123, -4315614.173286992)
('4d989f49b188721e56f72b37', "Pizoni's", -22.901800030699295, -43.35159634603115, 'R. Florianópolis, 1481, 21321-050', 208, True, -6239971.9150914885, -4315610.528102775)
('4f47d9d8e4b024e50b761712', 'Pizzonis', -22.90620630501871, -43.34803845646824, 'Brasil', 284, True, -6239051.9123150315, -4315958.107298673)
('540ba0dc498e1524a670d8dc', 'Pizzaria Barão 1400', -22.894792099263114, -43.35609797750852, 'Brasil', 326, True, -6241273.751473017, -4314954.662478668)
('4e707527ae602b2e7247f54c', 'Pizzaria Parada 1', -22.896515018229454, -43.35131897996615, 'Rua Cândido Benício, 1671, RJ, 21321-802', 233, True, -6240440.757113198, -4314789.154448158)
('578977bacd1045909d030fe4', 'Pizzaria Noventão', -22.893644, -43.34907, 'Rio de Janeiro, RJ, 2

In [17]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: Casa Nostra Pizzaria, Pizzaria e Sorveteria Sem Nome
Restaurants around location 103: Um Bom Lugar, Nunet's, Pensão do Gomes
Restaurants around location 104: Pensão do Gomes
Restaurants around location 105: Universidade das Pizzas
Restaurants around location 106: 
Restaurants around location 107: Restaurante Stambul, Restaurante do Kinha, Giraffas
Restaurants around location 108: 
Restaurants around location 109: 
Restaurants around location 110: Reis Da Souto, Bar do Zé


Let's now see all the collected restaurants in our area of interest on map, and let's also show Pizza's restaurants in different color.

In [18]:
map_rio = folium.Map(location=rio_center, zoom_start=13)
folium.Marker(rio_center, popup='Madureira').add_to(map_rio)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_pizza = res[6]
    color = 'red' if is_pizza else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_rio)
map_rio

Looks good. So now we have all the restaurants in the area just a few kilometers from Madureira, and we know which pizzerias are! We also know which restaurants are exactly in the vicinity of each neighborhood candidate center.

This concludes the data collection phase - we are now ready to use this data for analysis and produce the report on the ideal locations for a new pizzeria!

## Methodology <a name="methodology"></a>

In this project, we will direct our efforts to detect areas of Madureira with low density of restaurants, mainly those with low number of pizzerias. We will limit our analysis to an area of ​​approximately 6 km around the city center.

In the first stage, we collected the **necessary data: location and type (category) of each restaurant 6 km from the center of Madureira**. We also **identified pizzerias** (according to Foursquare categorization).

The second step in our analysis will be the calculation and exploration of '**restaurant density** 'in different areas of Madureira- we will use **heat maps** to identify some promising areas near the center with a low number of restaurants in general (* and * no pizzeria nearby) and focus our attention on these areas.

In the third and final stage, we will focus on the most promising areas and, within them, we will create ** groups of locations that meet some basic requirements ** established in discussion with stakeholders: we will consider locations with **a maximum of two restaurants within 250 meters**, and we want locations **without pizzerias within 400 meters**. We will present the map of all these locations, but we will also create clusters (using **k-means clustering**) of these locations to identify zones / neighborhoods / general addresses that should be a starting point for the final exploration of the 'street level' and search for the ideal location of the location by the interested parties.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [19]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 1.2417582417582418


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,NO ADDRESS,-22.893220,-43.364895,-6.242654e+06,-4.315501e+06,5992.495307,0
1,NO ADDRESS,-22.895024,-43.361841,-6.242054e+06,-4.315501e+06,5840.376700,0
2,NO ADDRESS,-22.896828,-43.358786,-6.241454e+06,-4.315501e+06,5747.173218,0
3,NO ADDRESS,-22.898631,-43.355730,-6.240854e+06,-4.315501e+06,5715.767665,2
4,NO ADDRESS,-22.900435,-43.352675,-6.240254e+06,-4.315501e+06,5747.173218,4
5,NO ADDRESS,-22.902239,-43.349619,-6.239654e+06,-4.315501e+06,5840.376700,2
6,NO ADDRESS,-22.904043,-43.346563,-6.239054e+06,-4.315501e+06,5992.495307,1
7,NO ADDRESS,-22.888065,-43.367791,-6.243554e+06,-4.314981e+06,5855.766389,1
8,NO ADDRESS,-22.889868,-43.364737,-6.242954e+06,-4.314981e+06,5604.462508,0
9,NO ADDRESS,-22.891671,-43.361682,-6.242354e+06,-4.314981e+06,5408.326913,0


OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [20]:
distances_to_pizza_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in pizza_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_pizza_restaurant.append(min_distance)

df_locations['Distance to Pizza restaurant'] = distances_to_pizza_restaurant

In [21]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Pizza restaurant
0,NO ADDRESS,-22.893220,-43.364895,-6.242654e+06,-4.315501e+06,5992.495307,0,1483.997638
1,NO ADDRESS,-22.895024,-43.361841,-6.242054e+06,-4.315501e+06,5840.376700,0,952.051898
2,NO ADDRESS,-22.896828,-43.358786,-6.241454e+06,-4.315501e+06,5747.173218,0,574.940555
3,NO ADDRESS,-22.898631,-43.355730,-6.240854e+06,-4.315501e+06,5715.767665,2,688.992357
4,NO ADDRESS,-22.900435,-43.352675,-6.240254e+06,-4.315501e+06,5747.173218,4,302.345016
5,NO ADDRESS,-22.902239,-43.349619,-6.239654e+06,-4.315501e+06,5840.376700,2,335.141536
6,NO ADDRESS,-22.904043,-43.346563,-6.239054e+06,-4.315501e+06,5992.495307,1,457.368563
7,NO ADDRESS,-22.888065,-43.367791,-6.243554e+06,-4.314981e+06,5855.766389,1,1661.421547
8,NO ADDRESS,-22.889868,-43.364737,-6.242954e+06,-4.314981e+06,5604.462508,0,1619.964110
9,NO ADDRESS,-22.891671,-43.361682,-6.242354e+06,-4.314981e+06,5408.326913,0,1080.196040


In [22]:
print('Average distance to closest Pizza restaurant from each area center:', df_locations['Distance to Pizza restaurant'].mean())

Average distance to closest Pizza restaurant from each area center: 827.4786317077004


OK, so **on average Pizzas restaurant can be found within ~827m** from every area center candidate. That's fairly close, so we need to filter our areas carefully!

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. Also, let's show **borders of Madureira boroughs** on our map and a few circles indicating distance of 1km, 2km and 3km.

In [23]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

pizza_latlons = [[res[2], res[3]] for res in pizza_restaurants.values()]

In [24]:
from folium import plugins
from folium.plugins import HeatMap

map_rio = folium.Map(location=rio_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_rio) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_rio)
folium.Marker(rio_center).add_to(map_rio)
folium.Circle(rio_center, radius=1000, fill=False, color='white').add_to(map_rio)
folium.Circle(rio_center, radius=2000, fill=False, color='white').add_to(map_rio)
folium.Circle(rio_center, radius=3000, fill=False, color='white').add_to(map_rio)
map_rio

Let's create another heatmap map showing **heatmap/density of Pizza restaurants** only.

In [25]:
map_rio = folium.Map(location=rio_center, zoom_start=13)
HeatMap(pizza_latlons).add_to(map_rio)
folium.Marker(rio_center).add_to(map_rio)
folium.Circle(rio_center, radius=1000, fill=False, color='white').add_to(map_rio)
folium.Circle(rio_center, radius=2000, fill=False, color='white').add_to(map_rio)
folium.Circle(rio_center, radius=3000, fill=False, color='white').add_to(map_rio)
map_rio

### Madureira

Madureira is one of the most classic neighborhoods in Rio's suburbs, home to the samba and pagode circles, classic literature like Lima Barreto.

"My place
 It is surrounded by fight and sweat
 Hope in a better world
 And beer to celebrate

 My place
 It has its myths and Beings of Light
 It's very close to Osvaldo Cruz
 Cascadura, Vaz Lobo and Irajá

 My place
 It's smile, it's peace and pleasure
 Your name is sweet to say
 Madureira, ia laiá
 Madureira, iá laiá "


Popular with tourists, alternative and bohemian, but expanding and modern, relatively close to the city center and with good connections, madureira is the root of the people of Rio.

In [26]:
roi_x_min = rio_center_x - 2000
roi_y_max = rio_center_y + 1000
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_rio = folium.Map(location=roi_center, zoom_start=13)
HeatMap(restaurant_latlons).add_to(map_rio)
folium.Circle(roi_center, radius=5500, color='white', fill=True, fill_opacity=0.4).add_to(map_rio)
map_rio

Not bad - this covers very well all low density restaurant pockets

We will also create a new and denser grid of candidates for places restricted to our new region of interest (we will make our candidates for places approximately 100 m).

In [27]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

2261 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Pizzas restaurant**.

In [28]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_pizza_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, pizza_restaurants)
    roi_pizza_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [29]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Pizza restaurant':roi_pizza_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Pizza restaurant
0,-22.891888,-43.347872,-6.240404e+06,-4.313785e+06,2,367.686313
1,-22.892189,-43.347363,-6.240304e+06,-4.313785e+06,2,281.245342
2,-22.889827,-43.350393,-6.240954e+06,-4.313698e+06,0,645.483705
3,-22.890127,-43.349883,-6.240854e+06,-4.313698e+06,0,642.885726
4,-22.890428,-43.349374,-6.240754e+06,-4.313698e+06,0,577.098770
5,-22.890728,-43.348865,-6.240654e+06,-4.313698e+06,2,522.287205
6,-22.891029,-43.348355,-6.240554e+06,-4.313698e+06,2,482.208315
7,-22.891329,-43.347846,-6.240454e+06,-4.313698e+06,2,454.047472
8,-22.891630,-43.347337,-6.240354e+06,-4.313698e+06,2,374.252354
9,-22.891930,-43.346827,-6.240254e+06,-4.313698e+06,2,306.546150


OK. Let us now **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 250 meters**, and **no Pizza restaurants in radius of 400 meters**.

In [30]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Pizza restaurant']>=800)
print('Locations with no Italian restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no more than two restaurants nearby: 2170
Locations with no Italian restaurants within 400m: 914
Locations with both conditions met: 903


Let's see how this looks on a map.

In [31]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_rio = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_rio)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_rio)
folium.Marker(rio_center).add_to(map_rio)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rio) 
map_rio

Looks good. We now have several locations very close to Madureira. Any of these locations is a potential candidate for a new restaurant, at least based on local competition.

Now let's show these good places in a heat map form:

In [32]:
map_rio = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_rio)
folium.Marker(rio_center).add_to(map_rio)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rio)
map_rio

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Pizza restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [33]:
from sklearn.cluster import KMeans

number_of_clusters = 10

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_rio = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_rio)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_rio)
folium.Marker(rio_center).add_to(map_rio)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_rio) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rio)
map_rio

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [34]:
map_rio = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(rio_center).add_to(map_rio)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_rio)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_rio)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_rio) 
map_rio

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [35]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')

for lon, lat in cluster_centers:
    lat1 , lon1 = str(lat), str(lon)
    loc = lat1+', '+lon1
    addr = geolocator.reverse(loc)
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, rio_center_x, rio_center_y)
    print('{}{} => {:.1f}km from Madureira \n'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

Rua Ferraz, Morro do Fubá, Cascadura, Zona Norte do Rio de Janeiro, Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, 21311050, Brasil                                                 => 3.4km from Madureira 

Maternidade Herculano Pinheiro, 276, Avenida Ministro Edgard Romero, Área de Proteção Ambiental e Recuperação Urbana da Serra da Misericórdia, Madureira, Zona Norte do Rio de Janeiro, Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, 21360-200, Brasil                                                 => 0.6km from Madureira 

Rua Joaquim Pimenta, Campinho, Zona Norte do Rio de Janeiro, Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região M

This concludes our analysis. We created 10 addresses that represent centers of areas containing locations with a low number of restaurants and no pizzeria nearby, all areas being very close to the city center. Although the zones are shown on the map with a radius of ~ 500 meters (green circles), their shape is really very irregular and their centers / addresses should only be considered as a starting point for exploring neighborhoods in the area in search of restaurant locations. potential. Most of the areas are located in the neighborhoods of Campinho and Cascadura, which we identified as interesting because they are popular with tourists, very close to the city center and with good public transport connections.

## Results and Discussion <a name="results"></a>

Our analysis shows that, although there are a large number of restaurants in Madureira, there are low density pockets of restaurants very close to the city center. The highest concentration of restaurants was detected in the north, so we focused our attention on the south, southeast and east areas, corresponding to the Campino and Cascadura neighborhoods.
After directing our attention to this narrowest area of ​​interest, we first created a dense grid of candidates for locations (spaced 100 m apart); these locations were filtered so that those with more than two restaurants within a 250m radius and those with an Pizza restaurant closer to 400m were removed.

These site candidates were then grouped together to create zones of interest that contain the largest number of site candidates. Center addresses in these zones were also generated using reverse geocoding to be used as markers / starting points for more detailed local analyzes based on other factors.

The result of all of this is 10 zones containing the largest number of potential new restaurants based on the number and distance from existing locations - both restaurants in general and pizza restaurants in particular. This, of course, does not mean that these areas are, in fact, ideal places for a new restaurant! The purpose of this analysis was to provide information only about areas close to the center of Madureira, but not crowded with existing restaurants (especially pizza) - it is perfectly possible that there is a very good reason for the small number of restaurants in any of these areas, reasons that would make them unsuitable for a new restaurant, regardless of the lack of competition in the area. The recommended zones should therefore only be considered as a starting point for a more detailed analysis that could eventually result in a location that not only has no competition nearby, but also other factors taken into account and all other relevant conditions met. .

## Conclusion <a name="conclusion"></a>

The objective of this project was to identify areas of Madureira close to the center with a low number of restaurants (especially pizza restaurants), in order to help interested parties to narrow the search for the ideal location for a new pizzeria. When calculating the distribution of the density of the restaurant from Foursquare data, we first identify general neighborhoods that warrant further analysis and then generate an extensive collection of locations that satisfy some basic requirements relating to restaurants in the vicinity. The grouping of these sites was then carried out in order to create the main zones of interest (containing the largest number of potential sites) and the addresses of these zone centers were created to be used as starting points for the final exploration by the interested parties .

The final decision on the ideal location of the restaurant will be made by the stakeholders based on the specific characteristics of the neighborhoods and locations in each recommended area, taking into account additional factors such as the attractiveness of each location (proximity to the park or water), noise levels / proximity to main roads, real estate availability, prices, social and economic dynamics of each neighborhood, etc.